In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from torch import nn
from torch.nn import functional as F
import torch
import pickle
from keras.preprocessing import sequence
import string
import re

Using TensorFlow backend.


In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [0]:
def clean_special_chars(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def func(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    return func(data, punct)


def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")

    for key in mapping.keys():
      if key in text:
        text = text.replace(key, mapping[key])

    text = [word.replace("'s", "") if word.endswith("'s") else word for word in text.split()]
    text = ' '.join(text)
    return text


def replace_repeating_chars(text):
    for char in string.ascii_lowercase:
      pattern = char + '{2,}'
      text = re.sub(pattern, char, text)
    return text


def preprocess(data):
  data = data.lower()
  data = clean_contractions(data, contraction_mapping)
  data = clean_special_chars(data)
  #data = replace_repeating_chars(data)
  return data

In [0]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)   
        x = x.permute(0, 3, 2, 1)  
        x = super(SpatialDropout, self).forward(x)  
        x = x.permute(0, 3, 2, 1)  
        x = x.squeeze(2)  
        return x

In [0]:
class IdentityLSTM(nn.Module):
  
    def __init__(self, max_features, n_targets=5, embed_size=300, units=16, num_layers=2):
      
        super(IdentityLSTM, self).__init__()
        
        self.embedding = nn.Embedding(max_features+1, embed_size)
        self.embedding_dropout = SpatialDropout(0.2)
        
        self.lstm = nn.LSTM(embed_size, units, bidirectional=True, dropout=0.2,
                            num_layers=num_layers, batch_first=True)
    
        self.linear = nn.Linear(units*4, units)     
        self.linear_out = nn.Linear(units, n_targets)
        
        
    def forward(self, x):
      
        m_embed = self.embedding(x)  
        m_embed = self.embedding_dropout(m_embed)
        
        m_lstm, _ = self.lstm(m_embed)

        avg_pool = torch.mean(m_lstm, 1) # torch.Size([1, units*2])
        max_pool, _ = torch.max(m_lstm, 1) # torch.Size([1, units*2])
        m_conc = torch.cat((max_pool, avg_pool), 1) # torch.Size([1, units*4])
        m_conc_linear  = F.relu(self.linear(m_conc))    
        out = self.linear_out(m_conc_linear)   
        return out, 0

In [0]:
class ToxicLSTM(nn.Module):
  
    def __init__(self, max_features, n_targets=6, embed_size=300, units=64, num_layers=2):
      
        super(ToxicLSTM, self).__init__()

        self.identity_lstm = IdentityLSTM(max_features)
        
        self.lstm = nn.LSTM(embed_size, units, bidirectional=True, dropout=0.2,
                            num_layers=num_layers, batch_first=True)
    
        self.linear = nn.Linear(units * 4, units * 2)
        self.linear_out = nn.Linear(units * 2 + 5, 1)
        self.linear_aux_out = nn.Linear(units * 2 + 5, n_targets)
        
        
    def forward(self, x):

        identities, _ = self.identity_lstm(x)
      
        m_embed = self.identity_lstm.embedding(x)  
        m_embed = self.identity_lstm.embedding_dropout(m_embed)  
        m_lstm, _ = self.lstm(m_embed)

        avg_pool = torch.mean(m_lstm, 1) # torch.Size([1, units*2])
        max_pool, _ = torch.max(m_lstm, 1) # torch.Size([1, units*2])
        m_conc = torch.cat(( max_pool, avg_pool), 1) # torch.Size([1, units*4+5])
   
        m_linear = torch.cat((identities, self.linear(m_conc)), 1)
        m_linear = torch.relu(m_linear)
        #hidden = m_conc + m_conc_linear
        
        result = self.linear_out(m_linear)
        aux_result = self.linear_aux_out(m_linear)
        out = torch.cat([result, aux_result], 1)      

        return out, identities

In [0]:
tok_path = F"/content/gdrive/My Drive/pytorch/tokenizer_best.pickle"
with open(tok_path, 'rb') as f:
  tokenizer = pickle.load(f)

In [9]:
max_features = len(tokenizer.word_index) 
print('max_features: {}'.format(max_features))

max_features: 150000


In [10]:
model_path = F"/content/gdrive/My Drive/pytorch/model_lstm_toxicity.pt"
state_dict = torch.load(model_path, map_location='cpu')
print('ROC-AUC score(s): {}'.format(state_dict['roc_auc']))
print('Validation Loss: {}'.format(state_dict['loss']))
model = ToxicLSTM(max_features)
model.load_state_dict(state_dict['state_dict'])
model.eval()

ROC-AUC score(s): 0.9680310941580701
Validation Loss: 0.07392344654936121


ToxicLSTM(
  (identity_lstm): IdentityLSTM(
    (embedding): Embedding(150001, 300)
    (embedding_dropout): SpatialDropout(p=0.2)
    (lstm): LSTM(300, 16, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (linear): Linear(in_features=64, out_features=16, bias=True)
    (linear_out): Linear(in_features=16, out_features=5, bias=True)
  )
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (linear): Linear(in_features=256, out_features=128, bias=True)
  (linear_out): Linear(in_features=133, out_features=1, bias=True)
  (linear_aux_out): Linear(in_features=133, out_features=6, bias=True)
)

In [0]:
categories_toxic = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
categories_identity = ['race', 'religion', 'sexuality', 'gender', 'disability']

In [0]:
def getPredictions(comments, model):
  comments = list(map(preprocess, comments))
  comments = tokenizer.texts_to_sequences(comments)
  MAX_LEN = max([len(x) for x in comments])
  comments = sequence.pad_sequences(comments, maxlen=MAX_LEN)
  comments = torch.tensor(comments, dtype=torch.long)
  toxicity, identities = model(comments)
  toxicity = torch.sigmoid(toxicity).tolist()
  identities = torch.sigmoid(identities).tolist()
  toxicity = [dict(zip(categories_toxic, x)) for x in toxicity]
  identities = [dict(zip(categories_identity, x)) for x in identities]
  return toxicity, identities
  

In [0]:
comments = ['I will kill you', 'Proud member of LGBT community', 
            'fucking muslim', 'proud to be a muslim']

In [0]:
toxicity, identities = getPredictions(comments, model)

In [28]:
for comment, score in zip(comments, toxicity):
  print(comment)
  print(score)
  print()

I will kill you
{'target': 0.9714114665985107, 'severe_toxicity': 0.4973338842391968, 'obscene': 0.03671025112271309, 'identity_attack': 0.023082993924617767, 'insult': 0.10097718983888626, 'threat': 0.8975567817687988, 'sexual_explicit': 0.015554099343717098}

Proud member of LGBT community
{'target': 0.0025926765520125628, 'severe_toxicity': 0.002852322533726692, 'obscene': 0.0018730449955910444, 'identity_attack': 0.06270429491996765, 'insult': 0.014784839004278183, 'threat': 0.001669012475758791, 'sexual_explicit': 0.002662821440026164}

fucking muslim
{'target': 0.9998311996459961, 'severe_toxicity': 0.5174486637115479, 'obscene': 0.7958086133003235, 'identity_attack': 0.6911431550979614, 'insult': 0.750449001789093, 'threat': 0.01001792959868908, 'sexual_explicit': 0.05653193220496178}

proud to be a muslim
{'target': 0.06221650913357735, 'severe_toxicity': 0.015402466058731079, 'obscene': 0.0021988593507558107, 'identity_attack': 0.25550368428230286, 'insult': 0.0313794873654842

In [29]:
for comment, score in zip(comments, identities):
  print(comment)
  print(score)
  print()

I will kill you
{'race': 4.827248631045222e-05, 'religion': 0.00024556959397159517, 'sexuality': 3.2471340091433376e-05, 'gender': 0.0004187581653241068, 'disability': 7.396675937343389e-05}

Proud member of LGBT community
{'race': 0.005679203663021326, 'religion': 0.03317807987332344, 'sexuality': 0.9704618453979492, 'gender': 0.004055017605423927, 'disability': 0.0003533287672325969}

fucking muslim
{'race': 0.00185526127461344, 'religion': 0.9997912049293518, 'sexuality': 0.0372181236743927, 'gender': 0.0019051034469157457, 'disability': 0.0007742712623439729}

proud to be a muslim
{'race': 0.0034487051889300346, 'religion': 0.999908447265625, 'sexuality': 0.3413546085357666, 'gender': 0.004958670120686293, 'disability': 0.0009836775716394186}

